In [1]:
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_tweet(tweet):

    tokens = word_tokenize(tweet)


    tokens = [token.lower() for token in tokens]


    tokens = [re.sub(r'https?://\S+', '', token) for token in tokens]


    tokens = [re.sub(r'@[A-Za-z0-9_]+', '', token) for token in tokens]


    tokens = [token.translate(str.maketrans('', '', string.punctuation)) for token in tokens]


    tokens = [token for token in tokens if token.strip()]


    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]


    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]


    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]


    return tokens

tweet = "Just finished reading a great book! #excited"
processed_tweet = preprocess_tweet(tweet)
print(processed_tweet)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


['finish', 'read', 'great', 'book', 'excit']


In [2]:
import gensim.downloader as api


word_vectors = api.load("glove-wiki-gigaword-100")


vector = word_vectors["word"]


king_vector = word_vectors["king"]
print(king_vector)

[==================================================] 100.0% 128.1/128.1MB downloaded
[-0.32307  -0.87616   0.21977   0.25268   0.22976   0.7388   -0.37954
 -0.35307  -0.84369  -1.1113   -0.30266   0.33178  -0.25113   0.30448
 -0.077491 -0.89815   0.092496 -1.1407   -0.58324   0.66869  -0.23122
 -0.95855   0.28262  -0.078848  0.75315   0.26584   0.3422   -0.33949
  0.95608   0.065641  0.45747   0.39835   0.57965   0.39267  -0.21851
  0.58795  -0.55999   0.63368  -0.043983 -0.68731  -0.37841   0.38026
  0.61641  -0.88269  -0.12346  -0.37928  -0.38318   0.23868   0.6685
 -0.43321  -0.11065   0.081723  1.1569    0.78958  -0.21223  -2.3211
 -0.67806   0.44561   0.65707   0.1045    0.46217   0.19912   0.25802
  0.057194  0.53443  -0.43133  -0.34311   0.59789  -0.58417   0.068995
  0.23944  -0.85181   0.30379  -0.34177  -0.25746  -0.031101 -0.16285
  0.45169  -0.91627   0.64521   0.73281  -0.22752   0.30226   0.044801
 -0.83741   0.55006  -0.52506  -1.7357    0.4751   -0.70487   0.056939
 -0.

In [5]:
import pandas as pd

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'twitter-sentiment-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1340873%2F2231927%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240412%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240412T072810Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4801f355cae46f75fe3cf2b1c305e6b6218100414eb9afe70c66f809c1ab976c888dbf6ef8ccb85630470edd464a90e014ee9bedf945109d6c383fd4bfe4fb026f83dd6c27a95e1355194f3ba8add4bab84aea60d338e35243a38f69d04340ce3ccf404a5d15fc1ba89c7c63953d9ee3134441846f3adc494da77ebbfcc1cc61dfd73817d2359f114a2dd081b7b88f330694247d45da35239d233e768678c86f9a6b4a4a7e51874f2f597dbe77869d541999acec06a54c69c06c303fe6fc8b165dec8e785ae5b322e7c8936000b96c0a00435d5a29663148dddc8655ec96b74f298046224ba27ad4f93e877a9069b47f94ff53dfefab8622adc80ca68a3f08c2'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')

data = pd.read_csv('/kaggle/input/twitter-sentiment-dataset/Twitter_Data.csv')

[==================================================] 7966522 bytes downloaded
Downloaded and uncompressed: twitter-sentiment-dataset
Data source import complete.


In [12]:
import torch
import torch.nn as nn

input_data = data['clean_text']



class SentimentRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SentimentRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out


input_size = 100
hidden_size = 128
output_size = 3
num_epochs = 10
learning_rate = 0.001

r
model = SentimentRNN(input_size, hidden_size, output_size)



model.load_state_dict(torch.load('/content/Twitter_sentiment.pth'))



<All keys matched successfully>

In [14]:


x = input_data[2]
y = data['category'][2]

print(x)
print(y)



def text_to_vectors(tokens):
    vectors = []
    for token in tokens:
        if token in word_vectors:
            vectors.append(word_vectors[token])
    return vectors


preprocessed_data = text_to_vectors(preprocess_tweet(x))
model.eval()

final_tensor = [preprocessed_data]

input_tensor = torch.tensor(final_tensor)

with torch.no_grad():
    predictions = model(input_tensor)


print(predictions)

what did just say vote for modi  welcome bjp told you rahul the main campaigner for modi think modi should just relax
1.0
tensor([[-0.4364,  0.0197,  0.6060]])
